In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy as sp
import numpy as np
from scipy import odr


import datetime

from datetime import timedelta
from datetime import datetime
from datetime import date
from dateutil.relativedelta import *
from datetime import datetime, timedelta
pd.set_option('display.max_rows', 300)

In [2]:
#Make a function that makes all negative numbers 0.
def OnlyPos(num):
    if num>=0: return num
    if num<0: return 0
print(OnlyPos(-10))


0


In [3]:
#input start run time
start = pd.to_datetime('2022-01-01 1:00')
end   = pd.to_datetime('2022-12-31 23:00')


panelcount=10

#area in m^2
panelarea=1.64*panelcount
panelefficiency=0.18
cloudloss=0.55

timediff=end-start

#input photovotaic orientation in elivation and azimuth
photangle=38

photaz=150

#back side #photaz=330

#House Facing #photaz=240

# Due South #photaz=180

#point in the sky that the photovoltaic points at
photel=90-photangle



In [4]:
timediff
timediffhours=24*timediff.days+timediff.seconds/3600

In [5]:


def sunpos(when, location, refraction):
# Extract the passed data
    year, month, day, hour, minute, second, timezone = when
    latitude, longitude = location
# Math typing shortcuts
    rad, deg = math.radians, math.degrees
    sin, cos, tan = math.sin, math.cos, math.tan
    asin, atan2 = math.asin, math.atan2
# Convert latitude and longitude to radians
    rlat = rad(latitude)
    rlon = rad(longitude)
# Decimal hour of the day at Greenwich
    greenwichtime = hour - timezone + minute / 60 + second / 3600
# Days from J2000, accurate from 1901 to 2099
    daynum = (
        367 * year
        - 7 * (year + (month + 9) // 12) // 4
        + 275 * month // 9
        + day
        - 730531.5
        + greenwichtime / 24
    )
# Mean longitude of the sun
    mean_long = daynum * 0.01720279239 + 4.894967873
# Mean anomaly of the Sun
    mean_anom = daynum * 0.01720197034 + 6.240040768
# Ecliptic longitude of the sun
    eclip_long = (
        mean_long
        + 0.03342305518 * sin(mean_anom)
        + 0.0003490658504 * sin(2 * mean_anom)
    )
# Obliquity of the ecliptic
    obliquity = 0.4090877234 - 0.000000006981317008 * daynum
# Right ascension of the sun
    rasc = atan2(cos(obliquity) * sin(eclip_long), cos(eclip_long))
# Declination of the sun
    decl = asin(sin(obliquity) * sin(eclip_long))
# Local sidereal time
    sidereal = 4.894961213 + 6.300388099 * daynum + rlon
# Hour angle of the sun
    hour_ang = sidereal - rasc
# Local elevation of the sun
    elevation = asin(sin(decl) * sin(rlat) + cos(decl) * cos(rlat) * cos(hour_ang))
# Local azimuth of the sun
    azimuth = atan2(
        -cos(decl) * cos(rlat) * sin(hour_ang),
        sin(decl) - sin(rlat) * sin(elevation),
    )
# Convert azimuth and elevation to degrees
    azimuth = into_range(deg(azimuth), 0, 360)
    elevation = into_range(deg(elevation), -180, 180)
    

    
# Refraction correction (optional)
    if refraction:
        targ = rad((elevation + (10.3 / (elevation + 5.11))))
        elevation += (1.02 / tan(targ)) / 60
        
        
    #Data table for Elevation to Power
    data = {'Sun_Elevation': [90, 75, 60, 45, 30, 20, 15, 10, 5, 0], 
        'Power': [1040, 1030, 1000, 950, 840, 710, 620, 470, 260, 20]}
    df_ElevationPower = pd.DataFrame(data)

    poly_model = odr.polynomial(5)
    data = odr.Data(df_ElevationPower.Sun_Elevation, df_ElevationPower.Power)
    odr_obj = odr.ODR(data, poly_model)
    output = odr_obj.run() 
    poly = np.poly1d(output.beta[::-1])
    poly_y = poly(df_ElevationPower.Sun_Elevation)
    df_gen_Elevation = pd.DataFrame({'gen':range(101)})
    gen_y=poly(df_gen_Elevation)
    power= OnlyPos(round(poly(elevation),1))    
   
# Return azimuth and elevation in degrees
    return (round(azimuth, 2), round(elevation, 2), round(power,2))
def into_range(x, range_min, range_max):
    shiftedx = x - range_min
    delta = range_max - range_min
    return (((shiftedx % delta) + delta) % delta) + range_min
if __name__ == "__main__":
# Close Encounters latitude, longitude
    location = (47.299, 11.204,)
# Fourth of July, 2022 at 11:20 am MDT (-6 hours)
    when=(2022, 4, 4, 6, 42, 0, +2)
# Get the Sun's apparent location in the sky
    azimuth, elevation, power = sunpos(when, location, True)


In [6]:
#plt.plot(df_ElevationPower.Sun_Elevation,  df_ElevationPower.Power,'ro', label="input data")
#plt.plot(df_gen_Elevation, gen_y, label="polynomial ODR")

In [7]:
#year, month, day, hour, minute, second, time zone
when=(2022, 1, 13, 12, 0, 0, +2)
location = (47.299, 11.204,)

#The function
sun_posdata=sunpos(when, location, True)

In [8]:

start_year= start.year
start_month= start.month
start_day=start.day
start_hour=start.hour
start_minute=start.minute
start_second=start.second

#year, month, day, hour, minute, second, time zone 
when=(start_year,start_month,start_day,start_hour, start_minute, 0, +2)
location = (47.299, 11.204,)

#The function
sun_posdata=sunpos(when, location, True)



#For the first value
#convert the "when" tuple to data frame data
my_tuple = when
dt_date = datetime(*my_tuple) #Unpacks the tuple into datetime arguments

#use the sun position data and the photovoltaic panel position to find the angle between the two and the incident ratio of the sun´s power onto the panel

sunaz=    sun_posdata[0]
sunel=    sun_posdata[1]
sunpower= sun_posdata[2]

incidence_angle=round(180/math.pi*math.acos(math.sin(sunel/180*math.pi)*math.sin(photel/180*math.pi)+math.cos(sunel/180*math.pi)*math.cos(photel/180*math.pi)*math.cos(sunaz/180*math.pi-photaz
 /180*math.pi)),0)

incidence_ratio=OnlyPos(math.cos(incidence_angle/180*math.pi))


incident_power=incidence_ratio*sunpower*panelarea*panelefficiency*cloudloss


df_data=pd.DataFrame({'Date_Time':[dt_date],'Sun_Azimuth':[sunaz],'Sun_Elevation':[sunel],'Sun_Power':[sunpower],'Angle_SuntoPanel':[incidence_angle],'PowerRatio_SuntoPanel':[incidence_ratio], 'Incident_Power':[incident_power]})


#For following values loop
#convert the "when" tuple to data frame data


cont=start
l=0
total_incident_power=incident_power



if start.month==1: January_incident_power=incident_power
else: January_incident_power=0

if start.month==2: February_incident_power=incident_power
else: February_incident_power=0

if start.month==3: March_incident_power=incident_power
else: March_incident_power=0

if start.month==4: April_incident_power=incident_power
else: April_incident_power=0

if start.month==5: May_incident_power=incident_power
else: May_incident_power=0    

if start.month==6: June_incident_power=incident_power
else: June_incident_power=0 
    
if start.month==7: July_incident_power=incident_power
else: July_incident_power=0 

if start.month==8: August_incident_power=incident_power
else: August_incident_power=0 

if start.month==9: September_incident_power=incident_power
else: September_incident_power=0 
    
if start.month==10: October_incident_power=incident_power
else: October_incident_power=0
    
if start.month==11: November_incident_power=incident_power
else: November_incident_power=0

if start.month==12: December_incident_power=incident_power
else: December_incident_power=0
    
    
    
while l<timediffhours:  


 cont=cont+ timedelta(hours=1)
 cont_year= cont.year
 cont_month= cont.month
 cont_day=cont.day
 cont_hour=cont.hour
 cont_minute=cont.minute
 cont_second=cont.second

 when=(cont_year,cont_month,cont_day,cont_hour, cont_minute, 0, +2)
 sun_posdata=sunpos(when, location, True)

 my_tuple = when
 dt_date = datetime(*my_tuple) #Unpacks the tuple into datetime arguments

    
 #use the sun position data and the photovoltaic panel position to find the angle between the two and the incident ratio of the sun´s power onto the panel
 sunaz=    sun_posdata[0]
 sunel=    sun_posdata[1]
 sunpower= sun_posdata[2]

 incidence_angle=round(180/math.pi*math.acos(math.sin(sunel/180*math.pi)*math.sin(photel/180*math.pi)+math.cos(sunel/180*math.pi)*math.cos(photel/180*math.pi)*math.cos(sunaz/180*math.pi-photaz
 /180*math.pi)),0)



 incidence_ratio=OnlyPos(math.cos(incidence_angle/180*math.pi))
 incident_power=incidence_ratio*sunpower*panelarea*panelefficiency*cloudloss
 total_incident_power=total_incident_power+incident_power


 if  dt_date.month==1:    
    January_incident_power= January_incident_power+incident_power      
    
 if  dt_date.month==2:    
    February_incident_power= February_incident_power+incident_power        
    
 if  dt_date.month==3:    
    March_incident_power= March_incident_power+incident_power
        
 if  dt_date.month==4:    
    April_incident_power= April_incident_power+incident_power
    
 if  dt_date.month==5:    
    May_incident_power= May_incident_power+incident_power
    
 if  dt_date.month==6:    
    June_incident_power= June_incident_power+incident_power
    
 if  dt_date.month==7:    
    July_incident_power= July_incident_power+incident_power
    
 if  dt_date.month==8:    
    August_incident_power= August_incident_power+incident_power

 if  dt_date.month==9:    
    September_incident_power= September_incident_power+incident_power
    
 if  dt_date.month==10:    
    October_incident_power= October_incident_power+incident_power

 if  dt_date.month==11:    
    November_incident_power= November_incident_power+incident_power
    
 if  dt_date.month==12:    
    December_incident_power= December_incident_power+incident_power


 #appended = {'Date_Time': dt_date, 'Sun_Azimuth': sunaz, 'Sun_Elevation': sunel,'Sun_Power':sunpower,'Angle_SuntoPanel':incidence_angle,'PowerRatio_SuntoPanel':incidence_ratio, 'Incident_Power':incident_power}

 #df_data = df_data.append(appended, ignore_index = True)

 
 l=l+1


    


#make the date format shorter
i=len(df_data)
q=0
while i>0:         
    df_data.at[q, 'Date_Time'] =df_data.at[q,'Date_Time'].strftime(" %a %d-%m %H:%M")
    q=q+1
    i=i-1
df_data

,Date_Time,Sun_Azimuth,Sun_Elevation,Sun_Power,Angle_SuntoPanel,PowerRatio_SuntoPanel,Incident_Power
0,Sat 01-01 01:00,349.72,-65.45,0,163.0,0,0.0


In [9]:
dt_date.month

12

In [10]:
round(total_incident_power/1000)


3316

In [11]:
#Only applicable if making a dataframe
sum(df_data.Incident_Power)

0.0

In [12]:
print("January   ", round(January_incident_power/1000))
print("February  ", round(February_incident_power/1000))
print("March     ", round(March_incident_power/1000))
print("April     ", round(April_incident_power/1000))
print("May       ", round(May_incident_power/1000))
print("June      ", round(June_incident_power/1000))
print("July      ", round(July_incident_power/1000))
print("August    ", round(August_incident_power/1000))
print("September ", round(September_incident_power/1000))
print("October   ", round(October_incident_power/1000))
print("November  ", round(November_incident_power/1000))
print("December  ", round(December_incident_power/1000))

print("          Total kWh    ",round(total_incident_power/1000))

January    163
February   202
March      292
April      332
May        368
June       363
July       371
August     355
September  303
October    249
November   175
December   143
          Total kWh     3316


In [13]:
#Kwh in the time frame %18 efficient cells. %83 with cloud cover. 20 m^2 panels per day
#round(total_incident_power/1000*0.18*0.83*20/365)


27

In [14]:
#Kwh in the time frame %18 efficient cells. %83 with cloud cover. 20 m^2 panels per day
#round(December_incident_power/1000*0.18*0.83*20/31)

14